In [1]:
# Initial setup by importing needed modules
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
test = pd.read_csv('data/LoanStats3b.csv', header=1, low_memory=False)

In [3]:
# repeat all the data wrangling steps
test_drop = test.dropna(axis=1, how='all', thresh=40000).dropna(subset=['loan_amnt'])
test_drop.select_dtypes(include='float').fillna(0);
test_drop = test_drop.assign(loan_start_d=pd.to_datetime(test_drop.issue_d, format='%b-%Y'))
test_drop['yr_emp'] = test_drop['emp_length'].str[0:2].replace('< ',0).astype('float')
test_drop.earliest_cr_line = pd.to_datetime(test_drop.earliest_cr_line, format='%b-%Y')
test_drop['yr_credit']= (2013-test_drop.earliest_cr_line.dt.year).fillna(0)
test_drop['revol_util_dec'] = test_drop['revol_util'].str.replace(r'%',r'0').astype('float')/100
loan_dict = {'Fully Paid': 1, 'Charged Off': 0, 'Does not meet the credit policy. Status:Fully Paid': 1, 'Does not meet the credit policy. Status:Charged Off': 0 }
test_drop = test_drop.assign(target=test_drop.loan_status.map(loan_dict))

In [4]:
# capturing the needed features for machine learning validations
test_ml = test_drop[['purpose', 'yr_credit', 'dti', 'revol_util_dec', 'total_acc', 'addr_state', 'target']]

In [5]:
# export to a csv file 
test_ml.to_csv('data/lending_test.csv', index=False)